In [2]:
from jwt import JWT, jwk
import hashlib
import os
import requests
import uuid
from urllib.parse import urlencode, unquote

import json

# UPBIT

## 초기 설정

In [9]:
access_key, secret_key = "", ""
server_url = "https://api.upbit.com"

keys = json.load(open("./keys.json"))
access_key = keys["access"]
secret_key = keys["secret"].encode("utf-8")

jwk_secret = jwk.OctetJWK(key=secret_key)

del keys

## 전체 계좌 정보 불러오기

In [11]:
payload = {
    'access_key': access_key,
    'nonce': str(uuid.uuid4()),
}

jwt_instance = JWT()
jwt_token = jwt_instance.encode(payload, key=jwk_secret)

authorization = 'Bearer {}'.format(jwt_token)
headers = {
  'Authorization': authorization,
}

res = requests.get(server_url + '/v1/accounts', headers=headers)
res.json()

[{'currency': 'KRW',
  'balance': '5000',
  'locked': '0',
  'avg_buy_price': '0',
  'avg_buy_price_modified': True,
  'unit_currency': 'KRW'}]

## 주문 가능 정보 조회

In [ ]:
params = {
  'market': 'KRW-BTC'
}

query_string = unquote(urlencode(params, doseq=True)).encode("utf-8")

m = hashlib.sha512()
m.update(query_string)
query_hash = m.hexdigest()

payload = {
    'access_key': access_key,
    'nonce': str(uuid.uuid4()),
    'query_hash': query_hash,
    'query_hash_alg': 'SHA512',
}

jwt_token = jwt_instance.encode(payload, jwk_secret)
authorization = 'Bearer {}'.format(jwt_token)
headers = {
  'Authorization': authorization,
}

res = requests.get(server_url + '/v1/orders/chance', params=params, headers=headers)
res.json()

## 주문하기
[API docs](https://docs.upbit.com/reference/%EC%A3%BC%EB%AC%B8%ED%95%98%EA%B8%B0)

In [ ]:
params = {
  'market': 'KRW-BTC', # 마켓 ID
  'side': 'bid',       # 주문 종류: ["bid": 매수, "ask": 매도]
  'ord_type': 'limit', # 주문 타입: ["limit": 지정가 주문, "price": 시장가 매수, "market": 시장가 매도]
  'price': '100.0',    # 주문 가격: KRW-BTC 마켓에서 1000 -> 1BTC당 매도 1호가가 500 KRW인 경우 2BTC가 매수됨.
  'volume': '0.01',    # 주문량
}

query_string = unquote(urlencode(params, doseq=True)).encode("utf-8")

m = hashlib.sha512()
m.update(query_string)
query_hash = m.hexdigest()

payload = {
    'access_key': access_key,
    'nonce': str(uuid.uuid4()),
    'query_hash': query_hash,
    'query_hash_alg': 'SHA512',
}

jwt_token = jwt.encode(payload, secret_key)
authorization = 'Bearer {}'.format(jwt_token)
headers = {
  'Authorization': authorization,
}

# res = requests.post(server_url + '/v1/orders', json=params, headers=headers)
res.json()

'''
200:
  uuid: 주문 고유 ID
  side: 주문 종류
  ord_type: 주문 방식
  price: 주문 당시 화폐 가격
  state: 주문 상태
  market: 마켓의 유일키
  created_at: 주문 생성 시간
  volume: 입력된 주문량
  remaining_volume: 체결 후 남은 주문 양
  reserved_fee: 수수료로 예약된 비용
  remaining_fee: 남은 수수료
  paid_fee: 지불된 수수료
  locked: 거래에 할당된 비용
  executed_volume: 체결량
  trades_count: 해당 주문에 걸린 체결 수

4XX:
  error:
    error_name: 오류 이름
    message: 오류 메시지
'''

## 마켓 코드 조회

In [3]:
url = "https://api.upbit.com/v1/market/all?isDetails=false"
headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

resp = response.json()

In [6]:
for res in resp:
    if "market" in res and res["market"] == "KRW-BTC":
        print("!")

!


In [7]:
url = "https://api.upbit.com/v1/candles/minutes/1?market=KRW-BTC&count=1"
response = requests.get(url, headers=headers)

resp = response.json()

In [8]:
resp

[{'market': 'KRW-BTC',
  'candle_date_time_utc': '2023-06-21T07:40:00',
  'candle_date_time_kst': '2023-06-21T16:40:00',
  'opening_price': 37901000.0,
  'high_price': 37910000.0,
  'low_price': 37900000.0,
  'trade_price': 37910000.0,
  'timestamp': 1687333253201,
  'candle_acc_trade_price': 75455250.48063,
  'candle_acc_trade_volume': 1.99068173,
  'unit': 1}]

## 시세 불러오기

In [ ]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'

keys = json.load(open("./keys.json"))
cmc_key = keys["coin_market_cap_key"]

parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': cmc_key,
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

## BINANCE

### 데이터 가져오기